In [1]:
# query Terrascope archive using terracatalgoueclient
from terracatalogueclient import Catalogue
import datetime as dt

# processes only VV

# define name of result
name = "testRun_01"

catalogue = Catalogue()

In [2]:
# give one date, find closest acqusition before and after
start = dt.date(2021, 10, 1) # input
end = dt.date(2021, 10, 2)

print("[CATALOGUE]: starting search for scenes...")

# first search
cat = catalogue.get_products(
	"urn:eop:VITO:CGS_S1_SLC_L1",
	start = start,
	end = end
)

for prod in cat:
    print(prod.geometry)

[CATALOGUE]: starting search for scenes...
POLYGON ((7.605903 51.141376, 8.148488 52.759369, 4.297474 53.172043, 3.894983 51.550926, 7.605903 51.141376))
POLYGON ((7.129499 49.65831, 7.649 51.277603, 3.927435 51.687271, 3.53682 50.065575, 7.129499 49.65831))
POLYGON ((6.671979 48.173637, 7.171072 49.794495, 3.568414 50.201855, 3.188162 48.579235, 6.671979 48.173637))


In [1]:
import glob
from pyroSAR import identify
import stsa
import pandas as pd
import sys, os
from contextlib import contextmanager
import geopandas as gpd

@contextmanager
def suppress_stdout():
    # source http://thesmithfam.org/blog/2012/10/25/temporarily-suppress-console-output-in-python/
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [2]:
file_list = glob.glob("/data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/2021/10/05/*/*.zip")

# load master dataframe
scene_proc = pd.read_pickle("scene_processing.pkl")
# print(scene_proc)

# make list to add to dataframe
meta_list = []

for e in file_list:
    name = e[e.rfind("/")+1:len(e)-4] # extract product name from path
    if not name in scene_proc["name"].values:
        # print("found new scene")
        # use pyroSAR to extract metadata
        pyro = identify(e)

        # make a dict
        meta = {"name": name, "path": e, "sensor": pyro.sensor, 
                "polarizations": pyro.polarizations, "start": pyro.start, "stop": pyro.stop, 
                "mode": pyro.acquisition_mode, "product": pyro.product,
                "orbit_direction": pyro.orbit, "rel_orbit": pyro.orbitNumber_rel}

        # use stsa to create swath geometry
        stsa_geom = stsa.TopsSplitAnalyzer(target_subswaths=['iw1', 'iw2', 'iw3'])
        with suppress_stdout():
            stsa_geom.load_data(zip_path = e)
            stsa_geom._create_subswath_geometry()
            
        stsa_geom = stsa_geom.df
        stsa_geom["name"] = name
        stsa_geom["path"] = 
        print(stsa_geom)
        
        # check whether the scene has a regular burst pattern (no missing, no additional bursts)
        if stsa_geom.df.loc[:,"burst"].to_list() == [1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9]:
            meta["regular_burst_pattern"] = 1
        else:
            meta["regular_burst_pattern"] = 0

        # add swath/burst geometry to metadata
        meta["geometry"] = stsa_geom.df
        # append to list
        meta_list.append(meta)
        
        for i, row in scene_proc.iterrows():
            print(row["geometry"])
            # TODO calc overlap etc.
        
    else:
        pass
        # print("old scene")
    
    
    
# print(set(pd.DataFrame(meta_list)["rel_orbit"].to_list()))
# orbits of the year 2021 are {1, 161, 37, 101, 8, 137, 139, 110, 88, 59}
meta_df = pd.DataFrame(meta_list)
# scene_proc = scene_proc.append(meta_df)
scene_proc.to_pickle("scene_processing.pkl")

SyntaxError: invalid syntax (<ipython-input-2-39a3d795410a>, line 31)

In [8]:
def create_gpd_for_scene(path, id, name):
    # use pyroSAR to extract metadata
    pyro = identify(path)

    # use stsa to create swath geometry
    stsa_geom = stsa.TopsSplitAnalyzer(target_subswaths=['iw1', 'iw2', 'iw3'])
    with suppress_stdout():
        stsa_geom.load_data(zip_path = path)
        stsa_geom._create_subswath_geometry()

    # add attributes to bursts
    stsa_geom = stsa_geom.df
    stsa_geom["id"] = id
    stsa_geom["name"] = name
    stsa_geom["path"] = path
    stsa_geom["sensor"] = pyro.sensor
    #stsa_geom["polarizations"] = pyro.polarizations
    stsa_geom["start"] = pyro.start
    stsa_geom["stop"] = pyro.stop
    stsa_geom["mode"] = pyro.acquisition_mode
    stsa_geom["product"] = pyro.product
    stsa_geom["orbit_direction"] = pyro.orbit
    stsa_geom["rel_orbit"] = pyro.orbitNumber_rel

    # check whether the scene has a regular burst pattern (no missing, no additional bursts)
    if stsa_geom.loc[:,"burst"].to_list() == [1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9]:
        stsa_geom["regular_burst_pattern"] = 1
    else:
        stsa_geom["regular_burst_pattern"] = 0

    # TODO calc overlap etc.
        
    return(stsa_geom)

# introduce geojson
file_list = glob.glob("/data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/2021/10/03/*/*.zip")

# load main dataframe
if os.path.isfile("scene_processing.geojson"):
    all_scenes = gpd.read_file("scene_processing.geojson")
    create_new_file = False
else:
    print("scene_processing.geojson not found")
    create_new_file = True

new_list = []

for path in file_list:
    name = path[path.rfind("/")+1:len(path)-4] # extract product name from path
    id = name[len(name)-4:len(name)]
    
    if create_new_file:
        stsa_geom = create_gpd_for_scene(path, id, name)
        # append to geodataframe list
        new_list.append(stsa_geom)
    
    elif not id in all_scenes["id"].values:
        # print("found new scene")
        stsa_geom = create_gpd_for_scene(path, id, name)

        # append to geodataframe list
        new_list.append(stsa_geom)
        
    else:
        pass
        # print("old scene")
    
    
    
# print(set(pd.DataFrame(meta_list)["rel_orbit"].to_list()))
# orbits of the year 2021 are {1, 161, 37, 101, 8, 137, 139, 110, 88, 59}
new_df = gpd.GeoDataFrame(pd.concat(new_list, ignore_index=True), crs=new_list[0].crs)
if not create_new_file:
    all_scenes = all_scenes.append(new_df)
else:
    all_scenes = new_df

all_scenes.to_file("scene_processing.geojson", driver='GeoJSON')
#scene_proc.to_pickle("scene_processing.pkl")

In [19]:
if os.path.isfile("scene_processing.geojson"):
    scene_proc = gpd.read_file("scene_processing.geojson")
    scene_proc
else:
    print("file not found")
    print(scene_procd)

file not found


NameError: name 'scene_procd' is not defined

In [75]:

            
print("hi")
with suppress_stdout():
    print("hio")

hi
